Osnabrück University - Machine Learning (Summer Term 2016) - Prof. Dr.-Ing. G. Heidemann, Ulf Krumnack

# Exercise Sheet 07

## Introduction

This week's sheet should be solved and handed in before the end of **Sunday, June 05, 2016**. If you need help (and Google and other resources were not enough), feel free to contact your groups designated tutor or whomever of us you run into first. Please upload your results to your group's studip folder.

## Assignment 1: Perceptron Theory (x Points)

## Assignment 2: Multilayer Perceptron (8 Points)

### (1) Starting with a single Perceptron
In this exercise you will implement a multilayer perceptron as described in the lecture. We start with the basic building block: the perceptron [ML-07 Slide 31].

In [2]:
import numpy as np
import numpy.random as rnd

# TODO: Write the activation and output function.

# Activation function.
actFun = lambda d, w: d @ np.transpose(w)

# The output function is determining the output of the neuron (1 if >0, else -1).
outFun = lambda x: x > 0

# TODO: Write a function that generates weights.

def generateWeights(bias, dims):
    '''
    Generates weights with the given bias for a number
    of dimensions.
    '''
    W = np.append(bias, rnd.rand(1, dims))
    W.shape = (1, dims + 1)
    return W

####################################################
## Testing the perceptron with a concrete example ##
####################################################

# Dimension of the data.
dims = 12 

# Input is a row vector.
D = np.append(1, rnd.rand(1, dims))
D.shape = (1, dims + 1)

#weights are stored in a vector
W = generateWeights(-1, dims)

out = outFun(actFun(D, W))
assert out == 1 or out == -1, "The output has to be either 1 or -1, but was %d" %out
assert actFun(D, W).shape == (1, 1), "The activation functions output should be one value"

Now we will use the above defined functions to train the perceptron to one of the following logical functions: AND, OR, NAND or NOR.

In [3]:
def eval_network(t, D, W):
    '''
    This function takes the trained weights of a perceptron
    and the input data (D) as well as the correct target values (t)
    and computes the overall error rate of the perceptron.
    '''
    error = 0.0
    size = np.max(D.shape)
    for i in range(0, size):
        out = outFun(actFun(D[i], W))
        error = error + np.abs(t[i] - out)
    # Normalize the error
    return error/size

In [7]:
#############################################
# Now we train our perceptron! [ML-07, sl.33]
#############################################

# TODO write the update function for the weights dependent
#      on epsilon, the target, the output and the input vector
delta_fun = lambda eps,t,y,x: eps*(t-y)*x


# Example learn one of the logical functions: AND, OR, NAND, NOR
# TODO define suitable parameters for your problem

eps = 0.1
dims = 2
training_size = 1250

W = generateWeights(-0.2, dims)

# Input
D_i = rnd.rand(training_size, dims)
D = np.ones((training_size, dims+1)) 
D[:,1:] = D_i #pad the input with ones for the bias
D = np.matrix(D)

# target function
op = lambda x1, x2: x1 and x2 #TODO change for other functions
log_op = lambda row: op(np.round(row[0]), np.round(row[1]))
labels = np.apply_along_axis(log_op,1,D[:,1:])

epochs = 1000
samp_size = 40

for i in range(0,epochs):
    error = 0
    #sample random from the training data
    for idx in rnd.choice(range(0, training_size), samp_size):
        y = outFun(actFun(D[idx], W))
        W = W + delta_fun(eps, labels[idx], y, D[idx])
        error = error + abs(labels[idx] - y)


#Print the overall performance of the Perceptron
print("Overall error of the Perceptron: {:.2%}".format(eval_network(labels, D, W)[0,0]))


Overall error of the Perceptron: 8.56%


In [143]:
import numpy as np

class Perceptron:
    def __init__(self, dimensions, eps=0.1):
        self.w = np.random.rand(1 + dimensions)
        self.eps = eps

    def evaluate(self, x):
        s = self.w @ np.append(1, x)
        return s > 0

    def adapt(self, x, y, t):
        self.w[np.newaxis] += self.eps * (t - y) * np.append(1, x)

    def train(self, x, t):
        y = self.evaluate(x)
        self.adapt(x, y, t)
        return y

    def __repr__(self):
        return "Perceptron({})".format(len(self.w) - 1)

In [144]:
import itertools
import numpy as np

X = np.matrix(list(itertools.product([0, 1], repeat=2)))

AND = lambda X : np.all(X, 1)
OR = lambda X : np.any(X, 1)
NAND = lambda X : 1 - AND(X)
NOR = lambda X : 1 - OR(X)
XOR = lambda X : X[:,0] != X[:,1]
NXOR = lambda X : 1 - XOR(X)

T = AND(X)
D = np.hstack((X, T))

In [145]:
p = Perceptron(2)
 
for s in np.random.choice(range(4), 100):
    x = D[s,0:2]
    t = D[s,2]
    y = p.train(x, t)

error = 0
for d in D:
    x = d[0,0:2]
    t = d[0,2]
    y = p.evaluate(x)
    error += np.abs(t - y) / len(D)

print("The perceptron has an error of {:.2%}".format(error))

The perceptron has an error of 0.00%


In [146]:
input_layer = [Perceptron(1), Perceptron(1)]
hidden_layer = [Perceptron(2)]
output_layer = [Perceptron(1)]

class MLP:
    def __init__(self, n_in, n_out, *n_hidden, eps=0.1):
        self.eps = eps
        self.layers = [[Perceptron(1) for _i in range(n_in)]]
        for layer in n_hidden:
            self.layers.append([Perceptron(len(self.layers[-1])) for _i in range(layer)])
        self.layers.append([Perceptron(len(self.layers[-1])) for _i in range(n_out)])

    def evaluate(self, x):
        self.o = [[p.evaluate(x[:,i]) for i, p in enumerate(self.layers[0])]]
        for i, layer in enumerate(self.layers[1:]):
            self.o.append([p.evaluate(self.o[i]) for p in layer])
        return self.o[-1]

    def adapt(self, t):
        # calculate error signals delta
        
        # output layer
        for i in range(len(self.layers[-1])):
            o = np.matrix(self.o[-1][i])
            self.layers[-1][i].delta = o * (1 - o) * (t - o)

        # other layers, from back to front
        for k in range(len(self.layers) - 2, -1, -1):
            for i in range(len(self.layers[k])):
                o = np.matrix(self.o[k][i])
                delta = o * (1 - o) * np.sum([n.w[i] * n.delta for n in self.layers[k+1]])
                self.layers[k][i].delta = delta
        
        # adapt weights
        for k in range(len(self.layers) - 1):
            for i in range(len(self.layers[k+1])):
                self.layers[k+1][i].w[np.newaxis] += self.eps * self.layers[k+1][i].delta * np.matrix(self.o[k][i])

    def train(self, x, t):
        y = self.evaluate(x)
        self.adapt(t)
        for l in self.layers:
            for p in l:
                print(p.w)
        print()

mlp = MLP(2,1,1)

for s in np.random.choice(range(4), 10):
    x = D[s,0:2]
    t = D[s,2]
    y = mlp.train(x, t)

error = 0
for d in D:
    x = d[0,0:2]
    t = d[0,2]
    y = mlp.evaluate(x)
    error += np.abs(t - y) / len(D)

print("The multilayer perceptron has an error of {[0]:.2%}".format(error))


[ 0.28090908  0.14436183]
[ 0.34964381  0.52136844]
[ 0.33178455  0.37598814  0.05941852]
[ 0.7288982   0.55305168]

[ 0.28090908  0.14436183]
[ 0.34964381  0.52136844]
[ 0.33178455  0.37598814  0.05941852]
[ 0.7288982   0.55305168]

[ 0.28090908  0.14436183]
[ 0.34964381  0.52136844]
[ 0.33178455  0.37598814  0.05941852]
[ 0.7288982   0.55305168]

[ 0.28090908  0.14436183]
[ 0.34964381  0.52136844]
[ 0.33178455  0.37598814  0.05941852]
[ 0.7288982   0.55305168]

[ 0.28090908  0.14436183]
[ 0.34964381  0.52136844]
[ 0.33178455  0.37598814  0.05941852]
[ 0.7288982   0.55305168]

[ 0.28090908  0.14436183]
[ 0.34964381  0.52136844]
[ 0.33178455  0.37598814  0.05941852]
[ 0.7288982   0.55305168]

[ 0.28090908  0.14436183]
[ 0.34964381  0.52136844]
[ 0.33178455  0.37598814  0.05941852]
[ 0.7288982   0.55305168]

[ 0.28090908  0.14436183]
[ 0.34964381  0.52136844]
[ 0.33178455  0.37598814  0.05941852]
[ 0.7288982   0.55305168]

[ 0.28090908  0.14436183]
[ 0.34964381  0.52136844]
[ 0.33178455

### (2) Building up ot the Multilayer Perceptron
In this exercise you will extend the model from above to a Multilayer Perceptron with several neurons aranged in several layers.

## Assignment 3: Activation Function and Backpropagation (x Points)